In [1]:
data_var = '2023-12-15'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8915,2023-12-15,Eua Nba,00:00,Portland Trail Blazers,Utah Jazz,1.54,2.73,224.5,1.82,2.11,-5.5,2.06,1.87,txVSOYc6,0.649351,0.366300,0.549451,0.473934,0.015651,0.2,0.4,NaN,NaN,569.508,223.436282,0.392332,436.756,259.082182,0.593197,379.946,410.594,459.20,880.80,1.0,1.0,0.0,0.0,0.394125,0.104357,0.068372,-2.75,-0.550,-0.981818,0.303098,0.2,-0.103098,-5.00,-1.000,-1.730000,0.295443,0.1,-0.195443
8916,2023-12-15,Eua Nba,00:00,Sacramento Kings,Oklahoma City Thunder,1.82,2.15,243.5,1.83,2.05,-2.5,2.04,1.88,bDPXNhDC,0.549451,0.465116,0.546448,0.487805,0.014567,0.8,0.4,NaN,NaN,221.386,20.649927,0.093276,193.428,27.408570,0.141699,229.090,209.600,191.88,183.82,0.0,0.0,0.0,0.0,0.117554,0.080187,0.057723,2.50,0.500,1.640000,0.545595,0.6,0.054405,-0.28,-0.056,-20.535714,0.511020,0.6,0.088980
8917,2023-12-15,Eua Nba,00:30,Los Angeles Clippers,Golden State Warriors,1.75,2.25,228.5,1.81,2.10,-3.5,2.09,1.85,x4OyNCSI,0.571429,0.444444,0.552486,0.476190,0.015873,0.0,0.8,NaN,NaN,206.974,43.559928,0.210461,273.330,55.956897,0.204723,172.556,283.080,198.73,233.16,0.0,0.0,0.0,0.0,0.176777,0.104891,0.086145,0.08,0.016,46.875000,0.711191,0.7,-0.011191,-5.00,-1.000,-1.250000,0.487323,0.3,-0.187323
8918,2023-12-15,Eua Nba,21:00,Charlotte Hornets,New Orleans Pelicans,3.18,1.43,230.5,1.80,2.09,5.5,2.09,1.85,ltyqV9Ku,0.314465,0.699301,0.555556,0.478469,0.013766,0.8,0.4,NaN,NaN,323.200,55.069296,0.170388,194.404,38.934479,0.200276,296.994,217.448,278.16,191.70,0.0,0.0,0.0,0.0,0.536849,0.105430,0.086145,-0.69,-0.138,-15.797101,0.438612,0.3,-0.138612,-0.83,-0.166,-2.590361,0.480820,0.4,-0.080820
8919,2023-12-15,Eua Nba,21:00,Philadelphia 76ers,Detroit Pistons,1.09,10.30,233.5,1.88,2.05,-16.5,2.02,1.92,2qumUTZo,0.917431,0.097087,0.531915,0.487805,0.014519,0.8,0.6,NaN,NaN,206.398,99.011862,0.479713,434.506,168.535540,0.387879,186.536,510.890,167.90,516.88,0.0,0.0,0.0,0.0,1.143539,0.061175,0.035894,-1.02,-0.204,-0.441176,0.687284,0.7,0.012716,-5.00,-1.000,-9.300000,0.244939,0.0,-0.244939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9031,2023-12-15,Finlândia Divisão I Feminina,14:00,HBA F,Tapiolan Honka II F,1.72,2.02,131.5,1.85,1.85,-1.5,1.85,1.85,ji48QhPc,0.581395,0.495050,0.540541,0.540541,0.076445,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,135.93,0.00,0.0,0.0,0.0,0.0,0.113440,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9032,2023-12-15,Kosovo Superliga Feminina,13:00,Peja F,Prishtina F,2.56,1.47,139.5,1.85,1.85,3.5,1.95,1.76,Uq4QhiPb,0.390625,0.680272,0.540541,0.540541,0.070897,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.382504,0.000000,0.072426,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9033,2023-12-15,Rússia Superliga,13:00,Tambov,MBA Moscow 2,1.48,2.53,153.5,1.85,1.85,-9.5,2.08,1.63,QVD3KaXS,0.675676,0.395257,0.540541,0.540541,0.070933,0.0,0.0,NaN,NaN,177.948,70.444416,0.395871,167.318,31.413166,0.187745,0.000,0.000,74.10,188.18,0.0,0.0,1.0,0.0,0.370305,0.000000,0.171535,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9034,2023-12-15,Suécia Liga De Basquete,14:30,Visby F,Lulea F,13.10,33.00,143.5,1.83,1.83,39.5,1.93,1.74,QmDSH6C0,0.076336,0.030303,0.546448,0.546448,-0.893361

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Portland Trail Blazers,Utah Jazz,224.5,1.82,1.0000,Over
1,00:00,Eua Nba,Sacramento Kings,Oklahoma City Thunder,243.5,1.83,0.9999,Over
2,00:30,Eua Nba,Los Angeles Clippers,Golden State Warriors,228.5,1.81,1.0000,Over
3,21:00,Eua Nba,Charlotte Hornets,New Orleans Pelicans,230.5,1.80,1.0000,Over
4,21:00,Eua Nba,Philadelphia 76ers,Detroit Pistons,233.5,1.88,1.0000,Over
5,21:00,Eua Nba,Washington Wizards,Indiana Pacers,256.5,1.86,1.0000,Over
6,21:30,Eua Nba,Boston Celtics,Orlando Magic,220.5,1.80,1.0000,Over
7,21:30,Eua Nba,San Antonio Spurs,Los Angeles Lakers,231.5,1.80,1.0000,Over
8,21:30,Eua Nba,Toronto Raptors,Atlanta Hawks,242.5,1.86,1.0000,Over
9,22:00,Eua Nba,Memphis Grizzlies,Houston Rockets,209.5,1.80,0.9999,Over
